In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import json
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV


def load_from_disk(name):
    try:
        with open(name, 'r') as f:
            tree = json.load(f)
            return tree
    except:
        return None

In [2]:
lbl2word = load_from_disk("generated/lbl2word.json")
word2lbl = load_from_disk("generated/word2lbl.json")
symbol2name = load_from_disk("generated/symbol2name.json")
name2symbol = load_from_disk("generated/name2symbol.json")

In [3]:
expcheb = pd.read_csv("generated/expcheb.csv")

In [4]:
expcheb.head()

,sName,sSymbol,eligble voters,voters,droped,valid,1,2,3,4,...,Arabs,Founding year,Current type of locality,Organizational affiliation,Coordinates,height,Planning Commission,Police space,year,Cluster Local Authorities
0,Tirosh,10,333,237,2,235,0,6,9,0,...,0.0,1955,310,2.0,1.892863e+09,130.0,151.0,15000597.0,2018,0.0
1,Mevasseret Ziyyon,1015,18871,13883,37,13846,0,2161,557,1,...,98.0,1951,160,0.0,2.144363e+09,583.0,152.0,10002475.0,2018,0.0
2,Me'ir Shefeya,102,106,54,0,54,0,11,4,0,...,38.0,1923,340,0.0,1.975772e+09,63.0,303.0,10004333.0,2018,0.0
3,Or Aqiva,1020,15520,9773,72,9701,0,323,499,0,...,93.0,1951,170,0.0,1.927371e+09,10.0,353.0,10004261.0,2018,0.0
4,Haruzim,1024,584,442,3,439,0,72,13,0,...,0.0,1951,350,0.0,1.874468e+09,39.0,401.0,15000060.0,2018,0.0


In [5]:
expcheb2 = pd.read_csv("generated/expcheb2.csv")

In [6]:
expcheb3 = pd.read_csv("generated/expcheb3.csv")

In [7]:
data = expcheb.append(expcheb2, ignore_index=True)

In [8]:
del expcheb2
del expcheb

In [9]:
def drop_cols(data):
    cols = ["sName", "voters", "valid", "droped", "subdistrict", "Natural area", "Metropolitan affiliation", "Founding year", "Organizational affiliation", "Coordinates", "height", "Planning Commission", "Police space", "year", "Cluster Local Authorities"]
    for col in cols:
        data = data.drop(col, axis=1)
        
    return data

In [10]:
train_label = data["droped"]
train_voters = data["voters"]
data = drop_cols(data)
test_label = expcheb3["droped"]
test_voters = expcheb3["voters"]
expcheb3 = drop_cols(expcheb3)

In [11]:
RSEED = 50

In [12]:
adb = AdaBoostRegressor()
adb_param_grid = {'n_estimators':[50,100,150,200,250], #Number of weak learners to train iteratively., 
                'learning_rate':[0.001, 0.01, 0.1, 1], #It contributes to the weights of weak learners. It uses 1 as a default value.,
                'random_state': [1]}

gsADB = GridSearchCV(adb,param_grid = adb_param_grid, cv=5, n_jobs= -1)

gsADB.fit(data,train_label)

ADB_best = gsADB.best_estimator_
    
print("Best Parameters:\n", gsADB.best_params_)
    
adb = AdaBoostRegressor(ADB_best)
adb.fit(data, train_label)
print(adb)

Best Parameters:
 {'learning_rate': 0.1, 'n_estimators': 250, 'random_state': 1}
AdaBoostRegressor(base_estimator=AdaBoostRegressor(base_estimator=None,
                                                   learning_rate=0.1,
                                                   loss='linear',
                                                   n_estimators=250,
                                                   random_state=1),
                  learning_rate=1.0, loss='linear', n_estimators=50,
                  random_state=None)


In [14]:
adb_predictions = adb.predict(expcheb3)
metrics.mean_squared_error(adb_predictions, test_label)

801.8068857045083

In [15]:
df = pd.DataFrame({"Predictions":adb_predictions})
names = [symbol2name[str(value)] for value in expcheb3["sSymbol"]]
df.insert(0,"sName", names)
df = df.sort_values('Predictions', ascending = False)
df

,sName,Predictions
493,Jerusalem,1868.533333
196,Bene Beraq,1234.188235
1187,Tel Aviv - Yafo,839.155172
405,Haifa,831.405063
114,Ashdod,808.187500
...,...,...
567,Kefar Ya'bez,11.922747
568,Kafar Kama,11.922747
137,Bahan,11.922747
570,Kefar Monash,11.922747
